In [1]:
#csv mit einem vorgegebenem Label generieren
#

In [ ]:
import yaml
import numpy as np
import time
from object_detection.utils import label_map_util
import os
import pathlib
import tensorflow as tf

with open("config.yaml", "r") as config_file:
    config = yaml.load(config_file, Loader=yaml.FullLoader)

PATH_TO_CHILD_DETECTED_IMAGES = "../images/detected_childs"
PATH_TO_ALL_IMAGES = config['PATH_TO_ALL_IMAGES']
PATH_TO_MODEL_DIR = config['PATH_TO_EXPORTED_MODEL']

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging (1)

tf.get_logger().setLevel('ERROR')  # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


def get_Image_Paths_Array(path):
    """
    @path ist ein pfad zu einem Ordner mit unterordner die Bilder Inhalten
    :returns ein Array mit den pfaden zu jedem Bild in den Unterordner
    """
    # assign directory
    #path = '/Volumes/Anas SSD/Bachelorarbeit/Bilder(Sammlung1)/'
    array = []
    for image in os.listdir(path):
        fullFileName = os.path.join(path, image)
        array.append(fullFileName)
    return array


IMAGE_PATHS = get_Image_Paths_Array(PATH_TO_ALL_IMAGES)
len(IMAGE_PATHS)

print('Loading model...', end='')
start_time = time.time()
# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_MODEL_DIR)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

import matplotlib

matplotlib.use("module://matplotlib_inline.backend_inline")

from PIL import Image
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')  # Suppress Matplotlib warnings


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))





In [ ]:
names_of_all_childs_images = []
for image_path in IMAGE_PATHS:
    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)
    if len(image_np.shape) == 2:
        image_np = np.stack((image_np,) * 3, axis=-1)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]
    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    ################################################################################################

    #copy all Child images into a python list
    classes = {"man": 1, "woman": 2, "child": 3}
    index_of_target_class = np.where(detections['detection_classes'] == classes.get("child"))[0]
    # count class 1 in detections['detection_classes'], write in column 'man' the number
    scores_of_target_class = []
    for index in index_of_target_class:
        score = detections['detection_scores'][index]
        scores_of_target_class.append(score)

    if any(num > 0.3 for num in scores_of_target_class):
        destinationPath = PATH_TO_CHILD_DETECTED_IMAGES
        os.system("cp " + "\"" + image_path + "\"" + " \"" + destinationPath + "\"")
        names_of_all_childs_images.append(image_path)

end_time = time.time()
print("###################################################################################################")
print("completed in", end_time - start_time)
# sphinx_gallery_thumbnail_number = 2

import pandas as pd

df = pd.DataFrame(names_of_all_childs_images)
df.to_csv('MY_CSV ' + time.time() + '.csv')
